In [321]:
import numpy as np

In [322]:
def initial_para(): # 初始化 维度为1 1
    w = np.zeros((1,1))
    b = np.zeros((1,1))
    dw = np.zeros((1,1))
    db =np.zeros((1,1))
    Vdw = np.zeros((1,1))
    Vdb = np.zeros((1,1))
    Sdw = np.zeros((1,1))
    Sdb = np.zeros((1,1))
    grads = {"dw": dw, "db": db}
    params = {"w": w, "b": b, "Vdw": Vdw, "Vdb": Vdb, "Sdw": Sdw, "Sdb":Sdb}
    return grads, params

def compute_loss(y, y_hat):
    n = y_hat.shape[1] #样本数量
    loss = np.squeeze(1 / (2 * n) * np.sum(np.square(y-y_hat), axis=1, keepdims=True))
    return loss

def forward_prop(X, params):
    w = params["w"]
    b = params["b"]
    y_hat = np.dot(w,X) + b 
    return y_hat

def back_prop(X, Y, y_hat, grads):
    n = X.shape[1]
    dz= - (Y - y_hat)
    dw = 1 / n * np.dot(dz, X.T)
    db = 1 / n * np.sum(dz, axis=1, keepdims=True)
    grads["dw"] = dw
    grads["db"] = db
    return grads

In [323]:
print(initial_para())
print(compute_loss(np.array([0.9, 2.1, 2.7, 3.1, 4.1, 4.8, 5.1, 5.9, 6.0, 7.0]).reshape(1,10),np.array([1,1,1,1,1,1,1,1,1,1]).reshape(1,10)))

({'dw': array([[0.]]), 'db': array([[0.]])}, {'w': array([[0.]]), 'b': array([[0.]]), 'Vdw': array([[0.]]), 'Vdb': array([[0.]]), 'Sdw': array([[0.]]), 'Sdb': array([[0.]])})
6.7195


In [324]:
def update_SGD(learning_rate, grads, params):
    dw = grads["dw"]
    db = grads["db"]
    w = params["w"]
    b = params["b"]
    
    w = w - learning_rate * dw
    b = b - learning_rate * db

    params["w"] = w 
    params["b"] = b
    return params

def update_Momentum(learning_rate, grads, params, beta1):
    dw = grads["dw"]
    db = grads["db"]
    w = params["w"]
    b = params["b"]
    Vdw = params["Vdw"]
    Vdb = params["Vdb"]

    Vdw = beta1 * Vdw + (1 - beta1) * dw
    Vdb = beta1 * Vdb + (1 - beta1) * db
    w = w - learning_rate * Vdw
    b = b - learning_rate * Vdb
    
    params["Vdw"] = Vdw
    params["Vdb"] = Vdb
    params["w"] = w 
    params["b"] = b
    return params

def update_RMSprop(learning_rate, grads, params, beta2, ε):

    dw = grads["dw"]
    db = grads["db"]
    w = params["w"]
    b = params["b"]
    Sdw = params["Sdw"]
    Sdb = params["Sdb"]

    Sdw = beta2 * Sdw + (1 - beta2) * (dw**2)
    Sdb = beta2 * Sdb + (1 - beta2) * (db**2)
    w = w - learning_rate * dw / (np.sqrt(Sdw) + ε)
    b = b - learning_rate * db / (np.sqrt(Sdb) + ε)

    params["Sdw"] = Sdw
    params["Sdb"] = Sdb
    params["w"] = w 
    params["b"] = b
    return params

def update_Adam(learning_rate, grads, params, beta2, beta1, iteration, ε):
    dw = grads["dw"]
    db = grads["db"]
    w = params["w"]
    b = params["b"]
    Vdw = params["Vdw"]
    Vdb = params["Vdb"]
    Sdw = params["Sdw"]
    Sdb = params["Sdb"]

    Vdw = beta1 * Vdw + (1 - beta1) * dw
    Vdb = beta1 * Vdb + (1 - beta1) * db
    Sdw = beta2 * Sdw + (1 - beta2) * (dw**2)
    Sdb = beta2 * Sdb + (1 - beta2) * (db**2)
    Vdw_co = Vdw / (1 - beta1**iteration) # **也可以
    Vdb_co = Vdb / (1 - beta1**iteration)
    Sdw_co = Sdw / (1 - beta2**iteration)
    Sdb_co = Sdb / (1 - beta2**iteration)
    w = w - learning_rate * Vdw_co / (np.sqrt(Sdw_co) + ε)
    b = b - learning_rate * Vdb_co / (np.sqrt(Sdb_co) + ε)

    params["Vdw"] = Vdw
    params["Vdb"] = Vdb
    params["Sdw"] = Sdw
    params["Sdb"] = Sdb
    params["w"] = w 
    params["b"] = b
    return params

In [325]:
X = np.array([0.0, 0.5, 0.8, 1.1, 1.5, 1.9, 2.2, 2.4, 2.6, 3.0]).reshape(1,10)
Y = np.array([0.9, 2.1, 2.7, 3.1, 4.1, 4.8, 5.1, 5.9, 6.0, 7.0]).reshape(1,10)
total_loss = 0
learning_rate = 0.01
grads, params = initial_para()
for i in range(100):
    t = np.random.randint(0, 9)
    y_hat = forward_prop(X[:,t].reshape(1,1), params)
    loss = compute_loss(Y[:,t].reshape(1,1), y_hat)
    grads = back_prop(X[:,t].reshape(1,1), Y[:,t].reshape(1,1), y_hat, grads)
    params = update_SGD(learning_rate, grads, params)
    print('第{}轮迭代，SGD的loss为{}'.format(i+1, loss))


第1轮迭代，SGD的loss为18.0
第2轮迭代，SGD的loss为3.1631155200000003
第3轮迭代，SGD的loss为3.979342777565389
第4轮迭代，SGD的loss为2.930411746426858
第5轮迭代，SGD的loss为13.617816267821796
第6轮迭代，SGD的loss为11.83891766048033
第7轮迭代，SGD的loss为10.320949800257628
第8轮迭代，SGD的loss为6.214383972724419
第9轮迭代，SGD的loss为7.816516016931413
第10轮迭代，SGD的loss为6.846671244102096
第11轮迭代，SGD的loss为0.12718401423619413
第12轮迭代，SGD的loss为0.1246530523528939
第13轮迭代，SGD的loss为2.9053336479296727
第14轮迭代，SGD的loss为5.542385865179033
第15轮迭代，SGD的loss为0.09544667691659552
第16轮迭代，SGD的loss为1.058528730467731
第17轮迭代，SGD的loss为1.0240936899957362
第18轮迭代，SGD的loss为2.9116991595542236
第19轮迭代，SGD的loss为0.07202947537432192
第20轮迭代，SGD的loss为2.612606111322325
第21轮迭代，SGD的loss为2.375591645798663
第22轮迭代，SGD的loss为0.8259770741615808
第23轮迭代，SGD的loss为2.0798219763672625
第24轮迭代，SGD的loss为2.6811971350486936
第25轮迭代，SGD的loss为0.6099295444357803
第26轮迭代，SGD的loss为2.519823563802673
第27轮迭代，SGD的loss为2.190658406905475
第28轮迭代，SGD的loss为0.27328914997552567
第29轮迭代，SGD的loss为0.4235525296111626
第30轮迭代，SGD的loss为

In [326]:
X = np.array([0.0, 0.5, 0.8, 1.1, 1.5, 1.9, 2.2, 2.4, 2.6, 3.0]).reshape(1,10)
Y = np.array([0.9, 2.1, 2.7, 3.1, 4.1, 4.8, 5.1, 5.9, 6.0, 7.0]).reshape(1,10)
learning_rate = 0.1
beta1 = 0.9
grads, params = initial_para()
for i in range(100):
    y_hat = forward_prop(X, params)
    loss = compute_loss(Y, y_hat)
    grads = back_prop(X, Y, y_hat, grads)
    params = update_Momentum(learning_rate, grads, params, beta1)
    print('第{}轮迭代，Momentum的loss为{}'.format(i+1, loss))

第1轮迭代，Momentum的loss为10.3895
第2轮迭代，Momentum的loss为9.5307292089604
第3轮迭代，Momentum的loss为8.03375949422402
第4轮迭代，Momentum的loss为6.186488530297546
第5轮迭代，Momentum的loss为4.284843421121528
第6轮迭代，Momentum的loss为2.5835546080640492
第7轮迭代，Momentum的loss为1.2628953628972461
第8轮迭代，Momentum的loss为0.4136276925566776
第9轮迭代，Momentum的loss为0.0390794627529669
第10轮迭代，Momentum的loss为0.07076222899264258
第11轮迭代，Momentum的loss为0.3924719244165814
第12轮迭代，Momentum的loss为0.8674290813388366
第13轮迭代，Momentum的loss为1.3635673172628426
第14轮迭代，Momentum的loss为1.7733026116820736
第15轮迭代，Momentum的loss为2.0256894859338024
第16轮迭代，Momentum的loss为2.090477251042041
第17轮迭代，Momentum的loss为1.97495847108552
第18轮迭代，Momentum的loss为1.7154770719541383
第19轮迭代，Momentum的loss为1.3659569496667279
第20轮迭代，Momentum的loss为0.9858367042641047
第21轮迭代，Momentum的loss为0.6294373576214266
第22轮迭代，Momentum的loss为0.338177449348251
第23轮迭代，Momentum的loss为0.1363292010242667
第24轮迭代，Momentum的loss为0.030315502382482193
第25轮迭代，Momentum的loss为0.010985408868942635
第26轮迭代，Momentum的loss为0.057

In [327]:
X = np.array([0.0, 0.5, 0.8, 1.1, 1.5, 1.9, 2.2, 2.4, 2.6, 3.0]).reshape(1,10)
Y = np.array([0.9, 2.1, 2.7, 3.1, 4.1, 4.8, 5.1, 5.9, 6.0, 7.0]).reshape(1,10)
learning_rate = 0.01
beta2 = 0.999
ε = 1e-08
grads, params = initial_para()
for i in range(100):
    y_hat = forward_prop(X, params)
    loss = compute_loss(Y, y_hat)
    grads = back_prop(X, Y, y_hat, grads)
    params = update_RMSprop(learning_rate, grads, params, beta2, ε)
    print('第{}轮迭代，RMSprop的loss为{}'.format(i+1, loss))


第1轮迭代，RMSprop的loss为10.3895
第2轮迭代，RMSprop的loss为6.80022810343681
第3轮迭代，RMSprop的loss为4.934703778532499
第4轮迭代，RMSprop的loss为3.732962212485701
第5轮迭代，RMSprop的loss为2.890125361233366
第6轮迭代，RMSprop的loss为2.271472733478878
第7轮迭代，RMSprop的loss为1.8044635325474911
第8轮迭代，RMSprop的loss为1.4452661009821028
第9轮迭代，RMSprop的loss为1.1653289569666727
第10轮迭代，RMSprop的loss为0.9450556757825587
第11轮迭代，RMSprop的loss为0.7704761410391363
第12轮迭代，RMSprop的loss为0.6313451289390702
第13轮迭代，RMSprop的loss为0.5199853270387739
第14轮迭代，RMSprop的loss为0.4305470259468468
第15轮迭代，RMSprop的loss为0.3585146711834222
第16轮迭代，RMSprop的loss为0.30036669574298397
第17轮迭代，RMSprop的loss为0.25333436222513284
第18轮迭代，RMSprop的loss为0.21522675167840638
第19轮迭代，RMSprop的loss为0.18430124646021143
第20轮迭代，RMSprop的loss为0.15916609978303864
第21轮迭代，RMSprop的loss为0.13870613496597164
第22轮迭代，RMSprop的loss为0.12202543574359961
第23轮迭代，RMSprop的loss为0.10840272332669253
第24轮迭代，RMSprop的loss为0.09725633980403858
第25轮迭代，RMSprop的loss为0.08811659279699374
第26轮迭代，RMSprop的loss为0.08060379838452855
第

In [328]:
X = np.array([0.0, 0.5, 0.8, 1.1, 1.5, 1.9, 2.2, 2.4, 2.6, 3.0]).reshape(1,10)
Y = np.array([0.9, 2.1, 2.7, 3.1, 4.1, 4.8, 5.1, 5.9, 6.0, 7.0]).reshape(1,10)
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
ε = 1e-08
grads, params = initial_para()
for i in range(1,1000):
    y_hat = forward_prop(X, params)
    loss = compute_loss(Y, y_hat)
    grads = back_prop(X, Y, y_hat, grads)
    # print(grads)
    params = update_Adam(learning_rate, grads, params, beta2, beta1, i, ε)
    # print(params)
    print('第{}轮迭代，Adam的loss为{}'.format(i, loss))

第1轮迭代，Adam的loss为10.3895
第2轮迭代，Adam的loss为10.264311600198777
第3轮迭代，Adam的loss为10.139906688147619
第4轮迭代，Adam的loss为10.016298446691085
第5轮迭代，Adam的loss为9.893499770594431
第6轮迭代，Adam的loss为9.771523234662343
第7轮迭代，Adam的loss为9.650381063359823
第8轮迭代，Adam的loss为9.530085102153377
第9轮迭代，Adam的loss为9.410646790768352
第10轮迭代，Adam的loss为9.292077138533596
第11轮迭代，Adam的loss为9.174386701957856
第12轮迭代，Adam的loss为9.057585564654346
第13轮迭代，Adam的loss为8.941683319700877
第14轮迭代，Adam的loss为8.826689054493915
第15轮迭代，Adam的loss为8.712611338126093
第16轮迭代，Adam的loss为8.599458211288763
第17轮迭代，Adam的loss为8.487237178674633
第18轮迭代，Adam的loss为8.3759552038306
第19轮迭代，Adam的loss为8.265618706388219
第20轮迭代，Adam的loss为8.15623356157889
第21轮迭代，Adam的loss为8.047805101922947
第22轮迭代，Adam的loss为7.940338120966878
第23轮迭代，Adam的loss为7.833836878930512
第24轮迭代，Adam的loss为7.728305110116418
第25轮迭代，Adam的loss为7.623746031926939
第26轮迭代，Adam的loss为7.520162355329948
第27轮迭代，Adam的loss为7.417556296612495
第28轮迭代，Adam的loss为7.315929590261845
第29轮迭代，Adam的loss为7.21528350281573
第30轮迭